In [ ]:
#Python code to randomize cost matrix; created by Jeremy!
import random

def cost(x):
    for row in x:
        for val in row:
            print(val,end=',')
        print()
    print()
# example array of task costs on different nodes
x = [
    [50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50],
    [40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40],
    [40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40],
    [10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10],
    ]

print('initial x:')
print('----------')
cost(x)

# pseudo-random seed init
# (for repeatability of random sequence)
#random.seed(42)

# thresholds for changing costs
MIN_COST_MULTIPLIER=0.5
MAX_COST_MULTIPLIER=2.0

for i in range(len(x)):
    for j in range(len(x[i])):
        x[i][j] = random.randint(int(x[i][j]*MIN_COST_MULTIPLIER), int(x[i][j]*MAX_COST_MULTIPLIER))


print('final x:')
print('----------')
cost(x)


In [ ]:
# First Version: This Python code has 3 data structures:
# cost matrix. values represents the cost of each task on each node. Randomized!
# task_sizes. different tasks have different requirnments (could be CPU or RAM but not sure). Randomized!
# node_capacities. here node capacities are the sum of (task_sizes) each node can handle.

from __future__ import print_function
import time
from ortools.linear_solver import pywraplp
import os
import random

def main():
  #-------------------------------------------
  #randomize code created by Jeremy;
  #import random;
  def cost(x):
    for row in x:
        for val in row:
            print(val,end=',')
        print()
    print()
  # example array of task costs on different nodes
  x = [
      [50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50],
      [40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40],
      [40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40],
      [10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10],
      ]

  print('initial x:')
  print('----------')
  cost(x)

  # pseudo-random seed init
  # (for repeatability of random sequence)
  # random.seed(42)

  # thresholds for changing costs
  MIN_COST_MULTIPLIER=0.5
  MAX_COST_MULTIPLIER=2.0

  for i in range(len(x)):
        for j in range(len(x[i])):
            x[i][j] = random.randint(int(x[i][j]*MIN_COST_MULTIPLIER), int(x[i][j]*MAX_COST_MULTIPLIER))

  print('final x:')
  print('----------')
  cost(x)

  #-------------------------------------------
  # begin Google-or Tool;
  solver = pywraplp.Solver('SolveAssignmentProblem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  start = time.time()

  edge_devices = ['192.168.1.224', '192.168.1.168', '192.168.1.182', '192.168.1.131']

  new_edge_devices = []
  e_devices='('
  
  #cost matrix. Randomised!
  cost = x
   
  # Task sizes. Different tasks have differet requirnements. Randomised!
  task_sizes = [10, 7, 3, 12, 15, 4, 11, 5, 10, 15, 13, 7, 10, 7, 3, 12, 15, 4, 11, 5, 10, 15, 13, 7, 10, 7, 3, 12, 15, 4, 11, 5]
  #task_sizes = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
  print (task_sizes)
  print ()  
   
  # Node_capacities. Sum of (task_sizes) each node can do. For example: Pi2 can only do total of 5; Pi3B cannt exceed 4. 
  total_size_max = [40,60,70,150]
  print ('Nodes capacities = ', total_size_max)
  
  num_workers = len(cost)
  num_tasks = len(cost[1])

  # Variables
  x = {}
  for i in range(num_workers):
    for j in range(num_tasks):
      x[i, j] = solver.IntVar(0, 1, 'x[%i,%i]' % (i, j))

  # Constraints
  # The total size of the tasks each worker takes on is at most total_size_max.
  for i in range(num_workers):
    solver.Add(solver.Sum([task_sizes[j] * x[i, j] for j in range(num_tasks)]) <= total_size_max[i])

  # Each task is assigned to at least one worker.
  for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

  solver.Minimize(solver.Sum([cost[i][j] * x[i,j] for i in range(num_workers)
                                                  for j in range(num_tasks)]))
  sol = solver.Solve()

  print('Minimum cost = ', solver.Objective().Value())
  print()
  final_Workers_IP=[0]*len(cost[1])
  for i in range(num_workers):
    for j in range(num_tasks):
      if x[i, j].solution_value() > 0:
        #print (i)
        #assign workers to array based on the task they were assigned
        #final_Workers_IP[j]=edge_devices[i] +' task ',j
        final_Workers_IP[j]='\''+edge_devices [i]+'\' '

        #string
        e_devices+='\''+edge_devices [i]+'\' '
        #return (edge_devices [i])
        print('Edge node', i,' assigned to task', j, '  Cost = ', cost[i][j])
        #print ('')
  print()
  end = time.time()
  print("Time = ", round(end - start, 4), "seconds")
  #print (new_edge_devices)
  e_devices = e_devices[:-1]
  e_devices+=')'
  #print(e_devices)
  finalIPsBashFormat='('
  for i in range(num_tasks):
      finalIPsBashFormat+=final_Workers_IP[i]
  finalIPsBashFormat= finalIPsBashFormat[:-1]
  finalIPsBashFormat+=')'
  print ()  
  print(finalIPsBashFormat)

if __name__ == '__main__':
  main()


In [ ]:
# Second Version: This Python code makes calculation based on the cost matrix itself. No task_sizes/requirnment here!
# Omit (task_sizes); calculation made based on cost matrix! Normal assingment problem without considereing task_requirnments!
# node capacities = number of tasks (how many tasks) each node can handle concurrently (e.g Pi can handle 20 tasks image processing)

# objective is to minimise the total cost!
# cost matrix shows that cost on node 0 incure high computations while on node 3 incure less computations

import random
import time
from ortools.linear_solver import pywraplp

def main():
    #-------------------------------------------
    #randomize code created by Jeremy;
    #import random;
    def cost(x):
        for row in x:
            for val in row:
                print(val,end=',')
        print()
    print()
    
    # example array of task costs on different nodes
    x = [
      [50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50],
      [40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40],
      [40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40],
      [10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10],
      ]
    
    print('initial x:')
    print('----------')
    cost(x)
    
    #cost = [
    #       [39,28,60,56,53,42,38,94,36,100,79,29,28,36,52,54,89,28,96,50,94,78,53,82,100,60,25,45,79,68,60,44],
    #       [33,68,41,26,25,44,26,42,74,42,58,36,71,22,66,49,54,27,79,44,25,55,38,73,60,59,76,75,43,56,32,65],
    #       [24,22,62,34,69,38,25,74,34,75,26,44,37,49,60,73,43,30,43,42,33,62,37,64,79,63,61,24,58,60,30,54],
    #       [12,10,19,17,13,12,15,6,12,6,15,17,13,7,11,15,11,20,17,19,9,13,9,12,13,18,17,16,12,9,20,7],
    #       ]
    
    # pseudo-random seed init
    # (for repeatability of random sequence)
    #random.seed(42)
    
    # thresholds for changing costs
    MIN_COST_MULTIPLIER=0.5
    MAX_COST_MULTIPLIER=2.0
    
    for i in range(len(x)):
        for j in range(len(x[i])):
            x[i][j] = random.randint(int(x[i][j]*MIN_COST_MULTIPLIER), int(x[i][j]*MAX_COST_MULTIPLIER))
            
    print('final x:')
    print('----------')
    cost(x)
    print ()
    
  #-------------------------------------------
    #begin Google-or Tool;
    # Data
    costs = x
    num_workers = len(costs)
    num_tasks = len(costs[0])
    
    # how many tasks each node can process concurrently! (e.g. Pi2 can handle up to 3 tasks; Pi3B up to 4;)
    # For example (if task is image processing; Pi2 can handle up to 3)
    node_cap = [3,6,8,15]
    
    # Solver
    # Create the mip solver with the SCIP backend.
    #solver = pywraplp.Solver.CreateSolver('MIP')
    
    solver = pywraplp.Solver('SolveAssignmentProblem',
                             pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    start = time.time()
    edge_devices = ['192.168.1.224', '192.168.1.168', '192.168.1.182', '192.168.1.131']
    new_edge_devices = []
    e_devices='('
    
    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Number of tasks assinged to each node less than the node capacitiy!
    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= node_cap[i])

    # Each task is assigned to exactly one worker.
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

    # Objective
    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs[i][j] * x[i, j])
    
    solver.Minimize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()
    print('Minimum cost = ', solver.Objective().Value())
    
    print()
    final_Workers_IP=[0]*len(costs[1])
    print()
    
    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total cost = ', solver.Objective().Value(), '\n')
        for i in range(num_workers):
            for j in range(num_tasks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    final_Workers_IP[j]='\''+edge_devices [i]+'\' '
                    e_devices+='\''+edge_devices [i]+'\' '
                    print('Edge node %d assigned to task %d.  Cost = %d' % (i, j, costs[i][j]))
        print()
        end = time.time()
        print("Time = ", round(end - start, 4), "seconds")
        #print (new_edge_devices)
        e_devices = e_devices[:-1]
        e_devices+=')'
        finalIPsBashFormat='('
        for i in range(num_tasks):
            finalIPsBashFormat+=final_Workers_IP[i]
        finalIPsBashFormat= finalIPsBashFormat[:-1]
        finalIPsBashFormat+=')'
        print ()  
        print(finalIPsBashFormat)
if __name__ == '__main__':
    main()

# Codes below generate the allocation for bash format

In [ ]:
# For bash format
# Second Version: This Python code makes calculation based on the cost matrix itself. No task_sizes/requirnment here!
# Omit (task_sizes); calculation made based on cost matrix! Normal assingment problem without considereing task_requirnments!
# node capacities = number of tasks (how many tasks) each node can handle concurrently (e.g Pi can handle 20 tasks image processing)

# objective is to minimise the total cost!
# cost matrix shows that cost on node 0 incure high computations while on node 3 incure less computations

import random
import time
from ortools.linear_solver import pywraplp

def main():
    #-------------------------------------------
    #randomize code created by Jeremy;
    
    # example array of task costs on different nodes
    x = [
      [50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50],
      [40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40],
      [40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40],
      [10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10],
      ]   
    
    # pseudo-random seed init
    # (for repeatability of random sequence)
    #random.seed(42)
    
    # thresholds for changing costs
    MIN_COST_MULTIPLIER=0.5
    MAX_COST_MULTIPLIER=2.0
    
    for i in range(len(x)):
        for j in range(len(x[i])):
            x[i][j] = random.randint(int(x[i][j]*MIN_COST_MULTIPLIER), int(x[i][j]*MAX_COST_MULTIPLIER))
    
  #-------------------------------------------
    #begin Google-or Tool;
    # Data
    costs = x
    num_workers = len(costs)
    num_tasks = len(costs[0])
    
    # how many tasks each node can process concurrently! (e.g. Pi2 can handle up to 3 tasks; Pi3B up to 4;)
    # For example (if task is image processing; Pi2 can handle up to 3)
    node_cap = [3,6,8,15]
    
    # Solver
    # Create the mip solver with the SCIP backend.
    #solver = pywraplp.Solver.CreateSolver('MIP')
    
    solver = pywraplp.Solver('SolveAssignmentProblem',
                             pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    start = time.time()
    edge_devices = ['192.168.1.224', '192.168.1.168', '192.168.1.182', '192.168.1.131']
    new_edge_devices = []
    e_devices='('
    
    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Number of tasks assinged to each node less than the node capacitiy!
    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= node_cap[i])

    # Each task is assigned to exactly one worker.
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

    # Objective
    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs[i][j] * x[i, j])
    
    solver.Minimize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()
    #print('Minimum cost = ', solver.Objective().Value())
    
    #print()
    final_Workers_IP=[0]*len(costs[1])
    #print()
    
    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total cost = ', solver.Objective().Value(), '\n')
        for i in range(num_workers):
            for j in range(num_tasks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    final_Workers_IP[j]='\''+edge_devices [i]+'\' '
                    e_devices+='\''+edge_devices [i]+'\' '
                    #print('Edge node %d assigned to task %d.  Cost = %d' % (i, j, costs[i][j]))
        #print()
        end = time.time()
        #print("Time = ", round(end - start, 4), "seconds")
        #print (new_edge_devices)
        e_devices = e_devices[:-1]
        e_devices+=')'
        finalIPsBashFormat='('
        for i in range(num_tasks):
            finalIPsBashFormat+=final_Workers_IP[i]
        finalIPsBashFormat= finalIPsBashFormat[:-1]
        finalIPsBashFormat+=')'
        #print ()  
        print(finalIPsBashFormat)
if __name__ == '__main__':
    main()

In [ ]:
# For bash format. we have tasks sizes or task requirnments!
# First Version: This Python code has 3 data structures:
# cost matrix. values represents the cost of each task on each node. Randomized!
# task_sizes. different tasks have different requirnments (could be CPU or RAM but not sure). Randomized!
# node_capacities. here node capacities are the sum of (task_sizes) each node can handle.

from __future__ import print_function
import time
from ortools.linear_solver import pywraplp
import os
import random

def main():
  #-------------------------------------------
  #randomize code created by Jeremy;
  #import random;
  x = [
      [50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50],
      [40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40],
      [40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40],
      [10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10],
      ]


  # pseudo-random seed init
  # (for repeatability of random sequence)
  #random.seed(42)

  # thresholds for changing costs
  MIN_COST_MULTIPLIER=0.5
  MAX_COST_MULTIPLIER=2.0

  for i in range(len(x)):
        for j in range(len(x[i])):
            x[i][j] = random.randint(int(x[i][j]*MIN_COST_MULTIPLIER), int(x[i][j]*MAX_COST_MULTIPLIER))

  #-------------------------------------------
  # begin Google-or Tool;
  solver = pywraplp.Solver('SolveAssignmentProblem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  start = time.time()

  edge_devices = ['192.168.1.224', '192.168.1.168', '192.168.1.182', '192.168.1.131']

  new_edge_devices = []
  e_devices='('
  
  #cost matrix. Randomised!
  cost = x
   
  # Task sizes. Different tasks have differet requirnements. Randomised!
  #task_sizes = [10, 7, 3, 12, 15, 4, 11, 5, 10, 15, 13, 7, 10, 7, 3, 12, 15, 4, 11, 5, 10, 15, 13, 7, 10, 7, 3, 12, 15, 4, 11, 5]
  task_sizes = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
  #print (task_sizes)
  #print ()  
   
  # Node_capacities. Sum of (task_sizes) each node can do. For example: Pi2 can only do total of 5; Pi3B cannt exceed 4. 
  total_size_max = [30,60,80,150]
  #print ('Nodes capacities = ', total_size_max)
  
  num_workers = len(cost)
  num_tasks = len(cost[1])

  # Variables
  x = {}
  for i in range(num_workers):
    for j in range(num_tasks):
      x[i, j] = solver.IntVar(0, 1, 'x[%i,%i]' % (i, j))

  # Constraints
  # The total size of the tasks each worker takes on is at most total_size_max.
  for i in range(num_workers):
    solver.Add(solver.Sum([task_sizes[j] * x[i, j] for j in range(num_tasks)]) <= total_size_max[i])

  # Each task is assigned to at least one worker.
  for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

  solver.Minimize(solver.Sum([cost[i][j] * x[i,j] for i in range(num_workers)
                                                  for j in range(num_tasks)]))
  sol = solver.Solve()

  print('Minimum cost = ', solver.Objective().Value())
  #print()
  final_Workers_IP=[0]*len(cost[1])
  for i in range(num_workers):
    for j in range(num_tasks):
      if x[i, j].solution_value() > 0:
        #print (i)
        #assign workers to array based on the task they were assigned
        #final_Workers_IP[j]=edge_devices[i] +' task ',j
        final_Workers_IP[j]='\''+edge_devices [i]+'\' '

        #string
        e_devices+='\''+edge_devices [i]+'\' '
        #return (edge_devices [i])
        #print('Edge node', i,' assigned to task', j, '  Cost = ', cost[i][j])
        #print ('')
  #print()
  end = time.time()
  #print("Time = ", round(end - start, 4), "seconds")
  #print (new_edge_devices)
  e_devices = e_devices[:-1]
  e_devices+=')'
  #print(e_devices)
  finalIPsBashFormat='('
  for i in range(num_tasks):
      finalIPsBashFormat+=final_Workers_IP[i]
  finalIPsBashFormat= finalIPsBashFormat[:-1]
  finalIPsBashFormat+=')'
  #print ()  
  print(finalIPsBashFormat)

if __name__ == '__main__':
  main()
